In [1]:
import keras
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping

Then we upload the required data set

In [2]:
lung_data = pd.read_csv('/content/Lung cancer dataset.csv')

we then observe the dataset, there are many features as we can see and most  features are answered with 1 and 2. 1 indicating yes and no

In [3]:
lung_data.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
0,M,65,1,1,1,2,2,1,2,2,2,2,2,2,1,NO
1,F,55,1,2,2,1,1,2,2,2,1,1,1,2,2,NO
2,F,78,2,2,1,1,1,2,1,2,1,1,2,1,1,YES
3,M,60,2,1,1,1,2,1,2,1,1,2,1,2,2,YES
4,F,80,1,1,2,1,1,2,1,2,1,1,1,1,2,NO


In [4]:
lung_data.tail()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
2995,F,71,2,1,1,2,2,1,1,1,1,2,1,1,2,NO
2996,F,75,1,2,1,1,1,2,2,2,2,1,1,2,1,NO
2997,F,62,2,2,2,1,2,2,2,2,1,1,2,2,2,YES
2998,M,30,1,1,2,2,2,2,2,2,2,1,2,1,2,YES
2999,M,40,1,2,2,1,1,1,2,2,2,1,1,1,1,YES


In [5]:
lung_data.shape

(3000, 16)

In [6]:
lung_data.isnull().sum()

GENDER                   0
AGE                      0
SMOKING                  0
YELLOW_FINGERS           0
ANXIETY                  0
PEER_PRESSURE            0
CHRONIC_DISEASE          0
FATIGUE                  0
ALLERGY                  0
WHEEZING                 0
ALCOHOL_CONSUMING        0
COUGHING                 0
SHORTNESS_OF_BREATH      0
SWALLOWING_DIFFICULTY    0
CHEST_PAIN               0
LUNG_CANCER              0
dtype: int64

We turn the categorical features into numerical in order to be able to feed it to our model

In [7]:
label_encoder = LabelEncoder()
lung_data['GENDER'] = label_encoder.fit_transform(lung_data['GENDER'])
lung_data['LUNG_CANCER'] = label_encoder.fit_transform(lung_data['LUNG_CANCER'])
lung_data.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
0,1,65,1,1,1,2,2,1,2,2,2,2,2,2,1,0
1,0,55,1,2,2,1,1,2,2,2,1,1,1,2,2,0
2,0,78,2,2,1,1,1,2,1,2,1,1,2,1,1,1
3,1,60,2,1,1,1,2,1,2,1,1,2,1,2,2,1
4,0,80,1,1,2,1,1,2,1,2,1,1,1,1,2,0


now we observe the coulumns that we are dealing with

In [8]:
lung_data_col = lung_data.columns

In [9]:
lung_data_col

Index(['GENDER', 'AGE', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY',
       'PEER_PRESSURE', 'CHRONIC_DISEASE', 'FATIGUE', 'ALLERGY', 'WHEEZING',
       'ALCOHOL_CONSUMING', 'COUGHING', 'SHORTNESS_OF_BREATH',
       'SWALLOWING_DIFFICULTY', 'CHEST_PAIN', 'LUNG_CANCER'],
      dtype='object')

we divide our dataset to predictors and the target. In this case i trained my mode to identify patients with lung cancer based on all the features in the dataset except lung cancer.

In [10]:
predictors = lung_data[lung_data_col[lung_data_col != 'LUNG_CANCER']]
target = lung_data['LUNG_CANCER']

In [11]:
target.head()

0    0
1    0
2    1
3    1
4    0
Name: LUNG_CANCER, dtype: int64

In [12]:
predictors.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN
0,1,65,1,1,1,2,2,1,2,2,2,2,2,2,1
1,0,55,1,2,2,1,1,2,2,2,1,1,1,2,2
2,0,78,2,2,1,1,1,2,1,2,1,1,2,1,1
3,1,60,2,1,1,1,2,1,2,1,1,2,1,2,2
4,0,80,1,1,2,1,1,2,1,2,1,1,1,1,2


here i will normalize my predictors in order to see how well my model does on normalized vs un normalized.

In [13]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN
0,0.990545,0.667697,-0.981995,-1.028232,-0.987906,1.001835,0.980687,-0.979379,0.986590,1.005180,1.017317,0.978726,1.024124,1.020715,-0.997171
1,-1.009209,-0.011478,-0.981995,0.972219,1.011904,-0.997836,-1.019354,1.020715,0.986590,1.005180,-0.982651,-1.021396,-0.976118,1.020715,1.002503
2,-1.009209,1.550624,1.017995,0.972219,-0.987906,-0.997836,-1.019354,1.020715,-1.013255,1.005180,-0.982651,-1.021396,1.024124,-0.979379,-0.997171
3,0.990545,0.328109,1.017995,-1.028232,-0.987906,-0.997836,0.980687,-0.979379,0.986590,-0.994515,-0.982651,0.978726,-0.976118,1.020715,1.002503
4,-1.009209,1.686459,-0.981995,-1.028232,1.011904,-0.997836,-1.019354,1.020715,-1.013255,1.005180,-0.982651,-1.021396,-0.976118,-0.979379,1.002503


In this part i need to identify the number of predictors i will be using for the model

In [14]:
num_of_col = predictors_norm.shape[1]

In [15]:
num_of_col

15

here we built the model:

*   15 nodes in the input layer
*   3 hidden layer with all having 10 nodes, and all of them use the relu activation function in order to get rid of the vanishing gradient problem.
*   the output layer consists of one node
*   as for the optimizer we used adam and for loss function we used binary crossentropy since we are dealing with classification for two values





In [21]:
def classification_model():
  model = Sequential()
  model.add(Dense(10, activation = 'relu', input_shape = (num_of_col,)))
  model.add(Dense(10, activation = 'relu'))
  model.add(Dense(10, activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy')
  return model

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 16)                256       
                                                                 
 dense_47 (Dense)            (None, 16)                272       
                                                                 
 dense_48 (Dense)            (None, 16)                272       
                                                                 
 dense_49 (Dense)            (None, 16)                272       
                                                                 
 dense_50 (Dense)            (None, 16)                272       
                                                                 
 dense_51 (Dense)            (None, 16)                272       
                                                                 
 dense_52 (Dense)            (None, 1)               

Un normalized data

In [22]:
model = classification_model()
X_train, X_test, y_train, y_test = train_test_split( predictors, target, test_size=0.2)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=300,  verbose=2)
scores = model.evaluate(X_test, y_test, verbose=2, callbacks=[early_stopping])

Epoch 1/300


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                160       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 391 (1.53 KB)
Trainable params: 391 (1.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


this model with normalized data

In [20]:
model = classification_model()
X_train, X_test, y_train, y_test = train_test_split( predictors_norm, target, test_size=0.2)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=300,  verbose=2)
scores = model.evaluate(X_test, y_test, verbose=2, callbacks=[early_stopping])

Epoch 1/300
75/75 - 1s - loss: 0.7059 - val_loss: 0.7030 - 1s/epoch - 15ms/step
Epoch 2/300
75/75 - 0s - loss: 0.6962 - val_loss: 0.7013 - 151ms/epoch - 2ms/step
Epoch 3/300
75/75 - 0s - loss: 0.6932 - val_loss: 0.7011 - 153ms/epoch - 2ms/step
Epoch 4/300
75/75 - 0s - loss: 0.6913 - val_loss: 0.7025 - 189ms/epoch - 3ms/step
Epoch 5/300
75/75 - 0s - loss: 0.6896 - val_loss: 0.7036 - 154ms/epoch - 2ms/step
Epoch 6/300
75/75 - 0s - loss: 0.6884 - val_loss: 0.7038 - 150ms/epoch - 2ms/step
Epoch 7/300
75/75 - 0s - loss: 0.6871 - val_loss: 0.7040 - 173ms/epoch - 2ms/step
Epoch 8/300
75/75 - 0s - loss: 0.6859 - val_loss: 0.7047 - 146ms/epoch - 2ms/step
Epoch 9/300
75/75 - 0s - loss: 0.6849 - val_loss: 0.7055 - 140ms/epoch - 2ms/step
Epoch 10/300
75/75 - 0s - loss: 0.6839 - val_loss: 0.7062 - 152ms/epoch - 2ms/step
Epoch 11/300
75/75 - 0s - loss: 0.6824 - val_loss: 0.7075 - 152ms/epoch - 2ms/step
Epoch 12/300
75/75 - 0s - loss: 0.6809 - val_loss: 0.7089 - 189ms/epoch - 3ms/step
Epoch 13/300
75

As we can observe the model with normalization had the value loss and the loss at the same rate almost. Which means both the training data and the validity data are almost at the same rate of accuracy. As for the normalized data we can observe a slight change in value loss and loss. The value

1.   As we can observe the model with normalization had the value loss and the loss at the same rate almost. Which means both the training data and the validity data are almost at the same rate of accuracy.
2.   As for the normalized data we can observe a slight change in value loss and loss. The value loss increased slightly and the loss decreased slightly. Which means that our model is doing good regarding the training process, but when we test our model on a another dataset we can obseve that it makes errors more.



**Conclusion:**
overall the normalizing data when we dont have alot of unqiue number isnt a right option. Meaning our data only consists of two numbers for each feature which tells us we dont have many classes so the right to option is not to normalize it.
